# Add weather data to get "number of climbable days"
## - temp above x below y, precipitation days
# Compare affordability/quality of life
## - unemployment? job growth?

In [63]:
import pandas as pd
import numpy as np
import string
import json
#from pandas.io.json import json_normalize
from pandas import json_normalize
import requests

# Imports for chart
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, plot, iplot

In [2]:
all_df = pd.read_csv('all_df.csv')
climb_df = pd.read_csv('climb_df.csv')
agg_df = pd.read_csv('agg_df.csv')

In [3]:
all_df.head()

,Unnamed: 0,Crags,Climbs,ST
0,0,BankHead Forest/ Thompson Creek Trail,2,AL
1,1,Bankhead Forest/Ship Rock,10,AL
2,2,bankhead forest/sipsey picnic area,1,AL
3,3,Cherokee Bluffs,32,AL
4,4,Chewacla State Park,18,AL


In [4]:
climb_df.head()

,ST,num,type
0,AL,1,Trad
1,AL,7,Trad
2,AL,0,Trad
3,AL,10,Trad
4,AL,0,Trad


In [5]:
agg_df.head()

,ST,Climbs,Crags,Aid,Alpine,Boulder,Ice,Mixed,Sport,Toprope,Trad,text
0,AL,1563,32,1,0,959,0,0,334,103,254,Crags: 32<br>Boulder: 959<br>Sport: 334<br>Top...
1,AK,2055,10,21,585,332,244,60,713,216,862,Crags: 10<br>Boulder: 332<br>Sport: 713<br>Top...
2,AZ,10244,3,91,4,2410,0,0,4225,445,3749,Crags: 3<br>Boulder: 2410<br>Sport: 4225<br>To...
3,AR,2023,4,1,0,508,0,0,1090,60,446,Crags: 4<br>Boulder: 508<br>Sport: 1090<br>Top...
4,KS,185,23,0,0,177,0,0,1,8,5,Crags: 23<br>Boulder: 177<br>Sport: 1<br>Topro...


In [21]:
# Exclude CA, CO ""
boulder_df = agg_df.sort_values(by = ['Boulder'], ascending = False)[2:]
boulder_df.head(10)

,ST,Climbs,Crags,Aid,Alpine,Boulder,Ice,Mixed,Sport,Toprope,Trad,text
20,MA,4551,6,2,3,3834,9,3,79,613,310,Crags: 6<br>Boulder: 3834<br>Sport: 79<br>Topr...
43,UT,14602,7,557,632,3412,197,44,5311,708,5668,Crags: 7<br>Boulder: 3412<br>Sport: 5311<br>To...
48,WI,4204,58,0,0,2570,46,3,180,1202,1109,Crags: 58<br>Boulder: 2570<br>Sport: 180<br>To...
2,AZ,10244,3,91,4,2410,0,0,4225,445,3749,Crags: 3<br>Boulder: 2410<br>Sport: 4225<br>To...
28,NH,5681,26,48,261,2307,333,62,993,278,2069,Crags: 26<br>Boulder: 2307<br>Sport: 993<br>To...
46,WA,7897,10,48,410,1991,81,64,3260,557,2618,Crags: 10<br>Boulder: 1991<br>Sport: 3260<br>T...
42,TX,3181,51,2,5,1763,0,0,1042,294,274,Crags: 51<br>Boulder: 1763<br>Sport: 1042<br>T...
37,PA,2872,6,5,19,1559,37,10,666,454,434,Crags: 6<br>Boulder: 1559<br>Sport: 666<br>Top...
38,RI,1332,20,0,0,1307,1,0,8,27,4,Crags: 20<br>Boulder: 1307<br>Sport: 8<br>Topr...
31,NY,4698,54,21,33,1276,344,98,337,1139,2283,Crags: 54<br>Boulder: 1276<br>Sport: 337<br>To...


<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 20 to 17
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ST       48 non-null     object
 1   Climbs   48 non-null     int64 
 2   Crags    48 non-null     int64 
 3   Aid      48 non-null     int64 
 4   Alpine   48 non-null     int64 
 5   Boulder  48 non-null     int64 
 6   Ice      48 non-null     int64 
 7   Mixed    48 non-null     int64 
 8   Sport    48 non-null     int64 
 9   Toprope  48 non-null     int64 
 10  Trad     48 non-null     int64 
 11  text     48 non-null     object
dtypes: int64(10), object(2)
memory usage: 4.9+ KB


### Looks like Massachusetts has a ton of boulders. Utah, Arizona, and Texas aren't surprising, but Wisconsin, and MA certainly are. 

In [26]:
data = dict(type='choropleth',
            colorscale = 'inferno',
            locations = boulder_df['ST'],
            z = boulder_df['Boulder'],
            locationmode = 'USA-states',
            text = boulder_df['text'],
            marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
            colorbar = {'title':"Number of Boulders"}
            ) 

layout = dict(title = 'Boulders By State (except CA, CO)',
              geo = dict(scope='usa',
                         showlakes = True,
                         lakecolor = 'rgb(85,173,240)')
             )

In [27]:
choromap = go.Figure(data = [data],layout = layout)

In [28]:
iplot(choromap)

In [7]:
all_df[all_df['ST'] == 'MA']

,Unnamed: 0,Crags,Climbs,ST
364,0,"Central, MA",143,MA
365,1,"Eastern, MA",1352,MA
366,2,Greater Boston,65,MA
367,3,North Shore,2635,MA
368,4,"South Shore, Cape, and Islands",98,MA
369,5,Western MA,258,MA


### The site has changed the layout for several states, breaking them up into sections that group many crags together by which part of the state they're in.  This renders the 'crag count' in the gathered data fairly useless. The URLs to get to the list of specific crags in each area is not iterable, so we'll have to manually check this state. 

### North Shore has the most climbs, and they are mostly boulders. The main destination here is Lynn Woods, here: https://www.mountainproject.com/area/105908096/lynn-woods . This location has 1246 documented boulders, which is a TON of problems.  I had no idea that New England had such a massive and well-documented boulder area. If you're a boulderer like me, this puts a very positive spin on MA, especially considering that Lynn Wods is only a 30 minute drive from Boston. 



In [72]:
lat = '42.489'
lon = '-70.992'
key = '200729984-06ddf5a57f5e0426e5e57deece8b3563'
maxDistance = '10'
maxResults = '500'
minDiff = 'V0'
maxDiff = 'V10'
url = 'https://www.mountainproject.com/data/get-routes-for-lat-lon?lat=' + lat + '&lon=' + lon + '&maxDistance=' + maxDistance + '&key=' + key + '&maxResults=' + maxResults
response = requests.get(url)

In [73]:
routes_json = response.json()
with open('lynnWoods.json', 'w') as f:
    json.dump(routes_json, f)

In [74]:
#routes_df = pd.DataFrame.from_dict(routes_json['routes'])
#routes_df.head()

In [75]:
#pd.read_json('mydata.json')

In [76]:
lynn_woods_df = json_normalize(routes_json['routes'])
lynn_woods_df.head()

,id,name,type,rating,stars,starVotes,pitches,location,url,imgSqSmall,imgSmall,imgSmallMed,imgMedium,longitude,latitude
0,106906051,Stone Warrior,Boulder,V5,4.3,35,,"[Massachusetts, North Shore, Lynn Woods, Stone...",https://www.mountainproject.com/route/10690605...,https://cdn2.apstatic.com/photos/climb/1189936...,https://cdn2.apstatic.com/photos/climb/1189936...,https://cdn2.apstatic.com/photos/climb/1189936...,https://cdn2.apstatic.com/photos/climb/1189936...,-70.9863,42.4908
1,114793072,Subway,Boulder,V3,4.5,29,,"[Massachusetts, North Shore, Lynn Woods, Stone...",https://www.mountainproject.com/route/11479307...,https://cdn2.apstatic.com/photos/climb/1182350...,https://cdn2.apstatic.com/photos/climb/1182350...,https://cdn2.apstatic.com/photos/climb/1182350...,https://cdn2.apstatic.com/photos/climb/1182350...,-70.9863,42.4908
2,106951986,Bear Grease,Boulder,V1,4.7,26,,"[Massachusetts, North Shore, Lynn Woods, Stone...",https://www.mountainproject.com/route/10695198...,https://cdn2.apstatic.com/photos/climb/1135920...,https://cdn2.apstatic.com/photos/climb/1135920...,https://cdn2.apstatic.com/photos/climb/1135920...,https://cdn2.apstatic.com/photos/climb/1135920...,-70.9827,42.4850
3,106952022,Holly the Happy Healer,Boulder,V3,4.3,34,,"[Massachusetts, North Shore, Lynn Woods, Stone...",https://www.mountainproject.com/route/10695202...,https://cdn2.apstatic.com/photos/climb/1111143...,https://cdn2.apstatic.com/photos/climb/1111143...,https://cdn2.apstatic.com/photos/climb/1111143...,https://cdn2.apstatic.com/photos/climb/1111143...,-70.9830,42.4905
4,106906062,Green Haze,Boulder,V7+,4.9,14,,"[Massachusetts, North Shore, Lynn Woods, Stone...",https://www.mountainproject.com/route/10690606...,https://cdn2.apstatic.com/photos/climb/1144273...,https://cdn2.apstatic.com/photos/climb/1144273...,https://cdn2.apstatic.com/photos/climb/1144273...,https://cdn2.apstatic.com/photos/climb/1144273...,-70.9863,42.4908


In [77]:
len(lynn_woods_df)

500

In [82]:
lynn_woods_df['rating'].value_counts()

V3             55
V2             49
V4             45
V1             31
V6             29
               ..
V0- R           1
V0-             1
5.11c V4 R      1
5.9 V0 PG13     1
5.10            1
Name: rating, Length: 74, dtype: int64